<a href="https://colab.research.google.com/github/chiyanglin-AStar/python_stock_study/blob/main/%E5%8F%B0%E8%82%A1%E6%AF%8F%E6%97%A5%E8%B3%87%E6%96%99_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 特定台股每日資料 
### ref : https://chenchenhouse.com/python/

In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os

In [2]:
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
listed

,有價證券代號,有價證券名稱,市場別,產業別,公開發行/上市(櫃)/發行日
1,1101,台泥,上市,水泥工業,1962/02/09
2,1102,亞泥,上市,水泥工業,1962/06/08
3,1103,嘉泥,上市,水泥工業,1969/11/14
4,1104,環泥,上市,水泥工業,1971/02/01
5,1108,幸福,上市,水泥工業,1990/06/06
...,...,...,...,...,...
971,9944,新麗,上市,其他業,2002/08/26
972,9945,潤泰新,上市,其他業,1992/04/30
973,9946,三發地產,上市,建材營造業,2013/09/17
974,9955,佳龍,上市,其他業,2008/01/21


In [3]:
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num

1      1101.TW
2      1102.TW
3      1103.TW
4      1104.TW
5      1108.TW
        ...   
971    9944.TW
972    9945.TW
973    9946.TW
974    9955.TW
975    9958.TW
Name: 有價證券代號, Length: 975, dtype: object

In [4]:
##讀寫成csv檔
def stock_data(stock_id,time_start,time_end) :
    days = 24 * 60 * 60    #一天有86400秒 
    initial = datetime.datetime.strptime( '1970-01-01' , '%Y-%m-%d' )
    start = datetime.datetime.strptime( time_start , '%Y-%m-%d' )
    end = datetime.datetime.strptime( time_end, '%Y-%m-%d' )
    period1 = start - initial
    period2 = end - initial
    s1 = period1.days * days
    s2 = period2.days * days
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=" + str(s1) + "&period2=" + str(s2) + "&interval=1d&events=history&includeAdjustedClose=true"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}
    response = requests.get(url,headers = headers)
    df = pd.read_csv(StringIO(response.text),index_col = "Date",parse_dates = ["Date"])
    address = r"C:\Users\adsad\OneDrive\Desktop\stock\\" + stock_id + ".csv"
    if  os.path.isfile(address):
        df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
        if time_start not in df_new.index:
            df_new = df_new.append(df)
            df_new.to_csv(address,encoding='utf-8')
            print("已更新到最新資料")
        else:
            print("已是最新資料，無需更新")
    else:
        df.to_csv(address,encoding='utf-8')
        print("此為新資料，已創建csv檔")

In [ ]:
time_start = "2000-01-01"
time_end = "2020-12-23"
for i in stock_num :   
    try:
        stock_data(i, time_start,time_end)
        print(i + "successful")
    except:
        print(i + "fail")

In [ ]:
today=datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
time_start = str(today)
time_end = str(tomorrow)
for i in stock_num :   
    try:
        stock_data(i, time_start,time_end)
        print(i + "successful")
    except:
        print(i + "fail")